In [ ]:
import random
from itertools import combinations

# Define card ranks and suits

In [ ]:
RANKS = ['2', '3', '4', '5', '6', '7', '8', '9', 'T', 'J', 'Q', 'K', 'A']
SUITS = ['H', 'D', 'C', 'S']

# Generate a deck of cards

In [ ]:
def generate_deck():
    return [rank + suit for rank in RANKS for suit in SUITS]